<a href="https://colab.research.google.com/github/ogut77/DataScience/blob/main/BoostingClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
from matplotlib import pyplot as plt
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
df = pd.read_csv(DATA_PATH + "telecom_churn.csv")

df["International plan"] = pd.factorize(df["International plan"])[0]
df["Voice mail plan"] = pd.factorize(df["Voice mail plan"])[0]
df["Churn"] = df["Churn"].astype("int")
states = df["State"]
y = df["Churn"]
X=df.drop(["State", "Churn"], axis=1)


In [2]:
X

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,0,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2
3329,68,415,0,1,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3
3330,28,510,0,1,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2
3331,184,510,1,1,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2


In [3]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=17
)



In [4]:
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(random_state=17)
tree.fit(X_train, y_train)
# make predictions for test data
tree_pred = tree.predict(X_test)
accuracy_score(y_test, tree_pred) 

0.92

In [5]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=17 )
forest.fit(X_train, y_train)
# make predictions for test data
forest_pred = forest.predict(X_test)
accuracy_score(y_test, forest_pred) 

0.95

In [6]:
# fit model no training data
from xgboost import XGBClassifier
model = XGBClassifier(random_state=17)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.951

In [16]:
import lightgbm as lgb
t0 = time()
lgb_model = lgb.LGBMClassifier(random_state=17)
lgb_model.fit(X_train, y_train)
lgb_time = time() - t0
accuracy_score(y_test, lgb_model.predict(X_test))

0.956

In [7]:
from sklearn.model_selection import GridSearchCV, cross_val_score
forest_params = {"max_depth": range(6, 10), "max_features": range(5, 10)}

forest_grid = GridSearchCV(model, forest_params, cv=5, n_jobs=-1, verbose=True)

forest_grid.fit(X_train, y_train)

forest_grid.best_params_, forest_grid.best_score_
accuracy_score(y_test, forest_grid.predict(X_test)) 

Fitting 5 folds for each of 20 candidates, totalling 100 fits


0.951

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from time import time
t0 = time()
gbm_model = GradientBoostingClassifier(random_state=17)
gbm_model.fit(X_train, y_train)
gbm_time = time() - t0
accuracy_score(y_test, gbm_model.predict(X_test))

0.954

In [14]:
forest_params = {"max_depth": range(2, 15,2), "max_features": range(5, 19,2)}

forest_grid = GridSearchCV(gbm_model, forest_params, cv=3, n_jobs=-1, verbose=True)

forest_grid.fit(X_train, y_train)


accuracy_score(y_test, forest_grid.predict(X_test)) 

Fitting 3 folds for each of 49 candidates, totalling 147 fits


0.95

In [15]:
print(forest_grid.best_params_, forest_grid.best_score_)

{'max_depth': 10, 'max_features': 11} 0.9511369614197377


In [17]:
forest_params = {"max_depth": range(2, 15,2), "max_features": range(5, 19,2)}

forest_grid = GridSearchCV(gbm_model, forest_params, cv=5, n_jobs=-1, verbose=True)

forest_grid.fit(X_train, y_train)

forest_grid.best_params_, forest_grid.best_score_
accuracy_score(y_test, forest_grid.predict(X_test)) 

Fitting 5 folds for each of 49 candidates, totalling 245 fits


0.956

In [18]:
print(forest_grid.best_params_, forest_grid.best_score_)

{'max_depth': 4, 'max_features': 9} 0.9541425039747822


In [ ]:
!pip install optuna


In [140]:
import optuna

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    x, y = X_train,y_train

    classifier_name = trial.suggest_categorical("classifier", ["GradientBoostingClassifier", "RandomForest","LightGBM","XGBoost"])
    if classifier_name == "GradientBoostingClassifier":
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         classifier_obj = sklearn.ensemble.GradientBoostingClassifier(random_state=17,  max_depth=max_depth, max_features=max_features )
    elif classifier_name == "GradientBoostingClassifier":
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         classifier_obj = sklearn.ensemble.RandomForestClassifier(random_state=17, max_depth=max_depth,max_features=max_features )
    elif classifier_name == "LightGBM":
         import lightgbm as lgb
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         
         classifier_obj  = lgb.LGBMClassifier(random_state=17,max_depth=max_depth,max_features=max_features)
    else:
         from xgboost import XGBClassifier
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         classifier_obj  = XGBClassifier(random_state=17,max_depth=max_depth,max_features=max_features)
   

    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

[I 2022-03-11 00:02:40,684] A new study created in memory with name: no-name-85c0d942-7403-4e62-a718-a03c9c05b33f
[I 2022-03-11 00:02:41,651] Trial 0 finished with value: 0.9472792660453329 and parameters: {'classifier': 'XGBoost', 'max_depth': 14, 'max_features': 7}. Best is trial 0 with value: 0.9472792660453329.
[I 2022-03-11 00:02:42,618] Trial 1 finished with value: 0.9477071636454669 and parameters: {'classifier': 'XGBoost', 'max_depth': 13, 'max_features': 3}. Best is trial 1 with value: 0.9477071636454669.
[I 2022-03-11 00:02:42,959] Trial 2 finished with value: 0.9507085124051705 and parameters: {'classifier': 'LightGBM', 'max_depth': 8, 'max_features': 5}. Best is trial 2 with value: 0.9507085124051705.
[I 2022-03-11 00:02:43,204] Trial 3 finished with value: 0.8954165329486666 and parameters: {'classifier': 'GradientBoostingClassifier', 'max_depth': 2, 'max_features': 2}. Best is trial 2 with value: 0.9507085124051705.
[I 2022-03-11 00:02:45,409] Trial 4 finished with value:

FrozenTrial(number=18, values=[0.956281658081144], datetime_start=datetime.datetime(2022, 3, 11, 0, 2, 55, 428677), datetime_complete=datetime.datetime(2022, 3, 11, 0, 2, 55, 812280), params={'classifier': 'LightGBM', 'max_depth': 12, 'max_features': 18}, distributions={'classifier': CategoricalDistribution(choices=('GradientBoostingClassifier', 'RandomForest', 'LightGBM', 'XGBoost')), 'max_depth': IntUniformDistribution(high=16, low=2, step=1), 'max_features': IntUniformDistribution(high=18, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=18, state=TrialState.COMPLETE, value=None)


In [134]:
study.best_params

{'classifier': 'LightGBM', 'rf_max_depth': 12, 'rf_max_feature': 14}

In [137]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(random_state=17,max_depth= 12,_max_features= 14)
lgb_model.fit(X_train,y_train)
print(accuracy_score(y_test, lgb_model.predict(X_test)))

0.956
